In [104]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [105]:
class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:,:-1])      #取所有行(所有样本)，每行取第一列到倒数第二列
        self.y_data = torch.from_numpy(xy[:,[-1]])     #取所有行，每行取最后一列，注意要用方括号括起，表明这是一个矩阵

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [106]:
dataset = DiabetesDataset(r'datasets/diabetes.csv')
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=0)   #此处多线程改为>0就总是报错，不知如何解决...

In [107]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(8, 6)   #将8维输入降维为6维
        self.linear2 = torch.nn.Linear(6, 4)   #...
        self.linear3 = torch.nn.Linear(4, 1)   #...为1维
        self.sigmoid = torch.nn.Sigmoid()      #nn.Sigmoid可以构建计算图，functional.sigmoid不行

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        return x

In [108]:
model = Model()

criterion = torch.nn.BCELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [109]:
if __name__ == '__main__':
    for epoch in range(100):
        for i, (inputs, labels) in enumerate(train_loader):   #迭代时，DataLoader对象会调用Dataset对象的__getitem__方法来获取数据样本
            y_pred = model(inputs)
            loss = criterion(y_pred, labels)
            print(epoch, i, loss.item())
            optimizer.zero_grad()
            optimizer.step()

0 0 22.212289810180664
0 1 22.65263557434082
0 2 22.568359375
0 3 22.60323143005371
0 4 22.771703720092773
0 5 22.392908096313477
0 6 22.74270248413086
0 7 23.35735321044922
0 8 22.563751220703125
0 9 22.6022891998291
0 10 22.203693389892578
0 11 22.654640197753906
0 12 22.32396125793457
0 13 22.861042022705078
0 14 22.689441680908203
0 15 22.597597122192383
0 16 23.0469970703125
0 17 22.608152389526367
0 18 22.30095672607422
0 19 22.840974807739258
0 20 22.96824073791504
0 21 22.779052734375
0 22 22.979713439941406
0 23 16.39872932434082
1 0 22.58626365661621
1 1 23.144084930419922
1 2 22.779380798339844
1 3 23.15799331665039
1 4 22.47847557067871
1 5 23.250762939453125
1 6 22.67241668701172
1 7 22.58108139038086
1 8 22.589452743530273
1 9 22.67544937133789
1 10 22.96913719177246
1 11 22.403968811035156
1 12 22.60310173034668
1 13 22.397775650024414
1 14 22.759754180908203
1 15 22.594219207763672
1 16 22.300968170166016
1 17 22.765117645263672
1 18 22.598600387573242
1 19 22.650089263